# 4.8 Grouping Data & Aggregating Variablees: Guided Practice
## -groupby() function to group dataframe by a given column (using 'product_name' here)
## -apply the agg() function to each group to obtain mean values for 'order_number' column
## -apply similar function using mean instead
## -multiple aggregations, ie: df.groupby('department_id').agg({'order_number':['mean','min','max']})
## -Aggregating with transform() - use results to create a flag (3 step process)
## -Deriving columns with loc()

## Import libraries and data

In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
#import path
path = r'/Users/nancykray/Desktop/Instacart Basket Analysis'

In [4]:
path

'/Users/nancykray/Desktop/Instacart Basket Analysis'

In [8]:
#import ords_prods_merged_derived
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged_derived.pkl'))

In [9]:
df = ords_prods_merge[:1000000]

### SHAPE & HEAD: first two functions you should complete on every datatframe!  double check the shape(), and then double check the column names & first couple of rows with head()

In [10]:
df.shape

(1000000, 19)

In [11]:
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,First Order,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy days,Average orders
2,473747,1,3,3,12,21.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy days,Most orders
3,2254736,1,4,4,7,29.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy days,Average orders
4,431534,1,5,4,15,28.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy days,Most orders


In [13]:
#use groupby function to group dataframe by a given column (using 'product_name' here)
df.groupby('product_name')

In [15]:
#performing a single aggregation: 
#split data into groups based on 'department_id'
#apply the agg() function to each group to obtain mean values for 'order_number' column
df.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


### This table lists all the possible “department_id” values from the current dataframe together with their corresponding “order_number” means—in other words, the average number of orders per user for each department ID  

### note that #4 (produce) has a mean of around 19. Conversely, the department with an ID of 17 (household), only has a mean of around 11. The insight you can glean from this, then, is that produce sells considerably more on average than household goods.

##### Back in the first Achievement, you learned that there are designated statistical tests for proving a significant difference between the means of different groups (e.g., the t-test) because they often can’t be spotted with the naked eye. It could be good to apply such a test here to confirm the statistical significance of any differences in mean values.

In [16]:
# Run similar function as above without use of agg(), using mean() instead
df.groupby('department_id')['order_number'].mean()

department_id
4     18.825780
7     17.472355
13    17.993423
14    19.246334
16    19.463012
17    11.294069
19    19.305237
20    17.599636
Name: order_number, dtype: float64

#### Just remember the key difference in syntax between the two methods: when using agg(), put the column you want to aggregate inside the parentheses of the agg() function as an argument. When using mean() (or any other standard aggregation function), simply index the column with square brackets, then follow it with the function you want to use after the dot.

In [18]:
#another way using dot function - but not recommended.  Use brackets as their only function in Python is indexing
df.groupby('department_id').order_number.mean()

department_id
4     18.825780
7     17.472355
13    17.993423
14    19.246334
16    19.463012
17    11.294069
19    19.305237
20    17.599636
Name: order_number, dtype: float64



## Performing multiple aggregations

### All it comes down to is adding more arguments to your code. You’ll be working again with the “order_number” column, this time producing not only the mean but also the min and max:

In [19]:
df.groupby('department_id').agg({'order_number':['mean','min','max']})

order_number        
                      mean min max
department_id                     
4                18.825780   1  99
7                17.472355   1  99
13               17.993423   1  99
14               19.246334   1  99
16               19.463012   1  99
17               11.294069   1  98
19               19.305237   1  99
20               17.599636   1  99

 #### these results above are not exciting as the min will always be 1 and it is capped at 99 by data engineers at Instacart

## Aggregating with transform() - use results to create a flag
### In this example, you’ll be creating a loyalty flag column in your ords_prods_merge dataframe. You explored the role of flags in the previous Exercise. To recap, they help you target specific groups within your data according to a set of conditions, allowing for segmented analysis.
### You’ve been asked to locate these loyalty customers at Instacart so that the business strategy team can employ some kind of bonus point program for them. You’ll do this by first using the transform() function, which will create a new column containing the maximum frequency of the “order_number” column, and then using the loc() function, which will create a second column containing a flag designating whether a customer is “loyal” or not.
### Now, let’s map this task onto the three-step process introduced earlier:
### 1.    Split the data into groups based on the “user_id” column.
### 2.    Apply the transform() function on the “order_number” column to generate the max orders for each user.
### 3.    Create a new column, “max_order,” into which you’ll place the results of your aggregation.



In [26]:
#use transform() function to get a new 'max_order' column

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

/var/folders/fb/2jrdrrb52mv7jbhxt0nbx_740000gn/T/ipykernel_37352/4166882784.py:3: FutureWarning: The provided callable <function max at 0x1124fd8a0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [27]:
#Check to see if the new column appears:

ords_prods_merge.head(15)

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,First Order,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy days,Average orders,10
2,473747,1,3,3,12,21.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy days,Most orders,10
3,2254736,1,4,4,7,29.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy days,Average orders,10
4,431534,1,5,4,15,28.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy days,Most orders,10
5,3367565,1,6,2,7,19.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10
6,550135,1,7,1,9,20.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10
7,3108588,1,8,1,14,14.0,Repeat Customer,196,2,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10
8,2295261,1,9,1,16,0.0,Repeat Customer,196,4,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10
9,2550362,1,10,4,8,30.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy days,Average orders,10


In [28]:
#print a head of 100 rows to further check the data

ords_prods_merge.head(100)

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,First Order,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy days,Average orders,10
2,473747,1,3,3,12,21.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy days,Most orders,10
3,2254736,1,4,4,7,29.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy days,Average orders,10
4,431534,1,5,4,15,28.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy days,Most orders,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3226575,360,1,5,12,NaN,First Order,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,3
96,1469869,377,3,5,17,3.0,Repeat Customer,196,9,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,3
97,1927023,387,2,4,10,22.0,Repeat Customer,196,3,0,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy days,Most orders,8
98,858092,420,4,1,19,30.0,Repeat Customer,196,2,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Average orders,22


In [29]:
#Change the limit so that we can see the 100 rows in the head function since this view above is limited
pd.options.display.max_rows = None

In [30]:
# Re-run head function to see all 100 rows now

In [32]:
ords_prods_merge.head(100)

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,First Order,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy days,Average orders,10
2,473747,1,3,3,12,21.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy days,Most orders,10
3,2254736,1,4,4,7,29.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy days,Average orders,10
4,431534,1,5,4,15,28.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy days,Most orders,10
5,3367565,1,6,2,7,19.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10
6,550135,1,7,1,9,20.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10
7,3108588,1,8,1,14,14.0,Repeat Customer,196,2,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10
8,2295261,1,9,1,16,0.0,Repeat Customer,196,4,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10
9,2550362,1,10,4,8,30.0,Repeat Customer,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy days,Average orders,10


#### (Note: having output this large (100 rows above) will inflate the size significantly.  You should go back later and delete this output after you have done your checks by: CELL MENU -> CURRENT OUTPUTS -> CLEAR)

## Deriving columns with loc():   With your new column ready to go, all that’s left is to create a flag that assigns a “loyalty” label to a user ID based on its corresponding max order value.
#### LOYAL CUSTOMER => MAX ORDERS >40
#### REGULAR CUSTOMER => MAX ORDERS >10, BUT >=40
#### NEW CUSTOMER => MAX ORDER<=10

In [34]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [36]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [37]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [38]:
# print the new 'loyalty_flag' column using value_coutns()
ords_prods_merge['loyalty_flag'].value_counts(dropna=False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

### now you will want to check the dataframe with the head function again, but you can index using df['column'] function to just pull the columns you want.  If you wanted to write a head() function that returned the first 60 rows of only the “user_id,” “loyalty_flag,” and “order_number” columns, you could format it as follows:

In [39]:
# as noted, use head function for columns of interest only
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(60)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10
